<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/06b-BERTEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a analizar los embeddings que devuelve BERT.

-----------------------

Tarea: entender todo el código y responder donde dice **PREGUNTA**

## Configuración del entorno

In [1]:
!pip install -q transformers datasets bertviz watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import torch

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from bertviz import head_view, model_view
from bertviz.neuron_view import show
from scipy.spatial.distance import cosine

In [3]:
%load_ext watermark

In [4]:
%watermark -vp transformers,datasets,pandas,numpy

Python implementation: CPython
Python version       : 3.11.12
IPython version      : 7.34.0

transformers: 4.51.3
datasets    : 3.5.0
pandas      : 2.2.2
numpy       : 2.0.2



## Qué mirás, BERT?

Como vimos, BERT fue entrenado para Masked Language Modeling (_aka_ [fill-mask](https://huggingface.co/tasks/fill-mask) en HF).

Vamos a ver cómo le va en eso.


In [5]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert = AutoModelForMaskedLM.from_pretrained(
    "bert-base-cased", output_attentions=True, output_hidden_states=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
print(bert_tokenizer.mask_token) # <mask>

[MASK]


In [7]:
input_mlm = [
    "These [MASK] are making me thirsty!",
    "These pretzels are making me [MASK]!",
    "These songs are making me [MASK]!",
    "I am [MASK]! I am without speech.",
    "[MASK] more soup for you! NEXT!",
    "I'm a [MASK] industrialist and philanthropist and a bicyclist."
]

In [8]:
def predict_mask(input_str):
    """Tomamos el camino largo en lugar de usar pipeline
    """
    inputs = bert_tokenizer(input_str, return_tensors="pt")
    mask_index = np.where(inputs['input_ids'] == bert_tokenizer.mask_token_id)
    # .eval() to set dropout and batch normalization layers to evaluation mode
    bert.eval()
    outputs = bert(**inputs)
    top_5_predictions = torch.softmax(outputs.logits[mask_index], dim=1).topk(5)
    for i in range(5):
        token = bert_tokenizer.decode(top_5_predictions.indices[0, i])
        prob = top_5_predictions.values[0, i]
        print(f" {i+1}) {token:<20} {prob:.3f}")

In [9]:
for x in input_mlm:
    print(x)
    out = predict_mask(x)
    print("-"*70)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


These [MASK] are making me thirsty!
 1) things               0.227
 2) people               0.098
 3) guys                 0.034
 4) men                  0.033
 5) thoughts             0.025
----------------------------------------------------------------------
These pretzels are making me [MASK]!
 1) sick                 0.579
 2) crazy                0.102
 3) mad                  0.050
 4) nervous              0.022
 5) cry                  0.017
----------------------------------------------------------------------
These songs are making me [MASK]!
 1) crazy                0.188
 2) mad                  0.104
 3) sick                 0.094
 4) cry                  0.059
 5) sing                 0.059
----------------------------------------------------------------------
I am [MASK]! I am without speech.
 1) not                  0.042
 2) nothing              0.033
 3) alone                0.031
 4) crying               0.027
 5) deaf                 0.026
--------------------------

**PREGUNTA** ¿Cómo genera el modelo las probabilidades de la palabra enmascarada?

También podemos analizar los hidden states y los attention scores.

Para esto está muy bueno [BertViz](https://github.com/jessevig/bertviz) pero también lo podemos hacer a mano.

In [10]:
print(bert)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

**PREGUNTA** ¿Qué información tiene el último hidden state del modelo?

In [11]:
# podemos consultar todos los pesos del modelo con:
state_dict = bert.state_dict()
list(state_dict.keys())[:5]

# o con named_parameters()

['bert.embeddings.word_embeddings.weight',
 'bert.embeddings.position_embeddings.weight',
 'bert.embeddings.token_type_embeddings.weight',
 'bert.embeddings.LayerNorm.weight',
 'bert.embeddings.LayerNorm.bias']

In [12]:
print("Token embeddings tensor shape:")
print(state_dict["bert.embeddings.word_embeddings.weight"].shape)
print("Position embeddings tensor shape:")
print(state_dict["bert.embeddings.position_embeddings.weight"].shape)

Token embeddings tensor shape:
torch.Size([28996, 768])
Position embeddings tensor shape:
torch.Size([512, 768])


**PREGUNTA** ¿Cómo se interpretan las dimensiones de los dos tensores anteriores?

In [13]:
input_str = '"I voted for Obama because he was most aligned with my values", Mary said.'

In [14]:
model_inputs = bert_tokenizer(input_str, return_tensors="pt")
bert.eval()
with torch.inference_mode():
    model_output = bert(**model_inputs)

In [15]:
print(f"# hidden states = {len(model_output.hidden_states)}")
# initial embeddings + 12 transf. blocks

# hidden states = 13


In [16]:
print("Size of each hidden state:")
print(model_output.hidden_states[1].shape) # (bsz, tokens, dim)

Size of each hidden state:
torch.Size([1, 20, 768])


In [17]:
print("Size of each attention tensor:")
print(model_output.attentions[0].shape) # (bsz, head, query_word, key_word)

Size of each attention tensor:
torch.Size([1, 12, 20, 20])


**PREGUNTA** ¿Cómo se interpreta el tamaño del tensor anterior?

Veamos cómo extraer los contextual word embeddings (CWE) -- sin el [feature extractor de HF](https://huggingface.co/tasks/feature-extraction).

In [18]:
print(type(model_output.hidden_states))
print(model_output.hidden_states[0].shape)

<class 'tuple'>
torch.Size([1, 20, 768])


In [19]:
def get_cwes(model_output):
    """Contextual embeddings como la suma de last 4 layers
    """
    # stack los 13 states en un solo tensor
    embeddings = torch.stack(model_output.hidden_states, dim=0)
    #print(embeddings.shape)
    # drop dimension de batches:
    embeddings = torch.squeeze(embeddings, dim=1)
    #print(embeddings.shape)
    # sum last 4 layers
    embeddings = embeddings[-4:].sum(dim=0)
    #print(embeddings.shape)
    return embeddings

def extract_bert_cwe(input_str, target_word):
    """Extract BERT CWE of a specific token in input_str
    """
    model_inputs = bert_tokenizer(input_str, return_tensors="pt")
    target_position = model_inputs.tokens().index(target_word)
    bert.eval()
    with torch.inference_mode():
        model_output = bert(**model_inputs)
    embedding = get_cwes(model_output)[target_position]
    return embedding

In [20]:
# analicemos la similitud de "values" en cada contexto
input_strings = [
    '"I voted for Obama because he was most aligned with my values", Mary said.',
    'Find the values of x and y in x+y=8',
    'I believe in the values of liberal democracy.',
]

In [21]:
word_embeddings = []
for input_ in input_strings:
    emb_ = extract_bert_cwe(input_, "values")
    word_embeddings.append(emb_)

In [22]:
len(word_embeddings)

3

In [23]:
cos_ = torch.cosine_similarity(word_embeddings[0], word_embeddings[1], dim=0).item()
print(f'Cosine sim. entre "values" de')
print(f"  {input_strings[0]}")
print(f"  {input_strings[1]}")
print(f"{cos_:.4f}")

Cosine sim. entre "values" de
  "I voted for Obama because he was most aligned with my values", Mary said.
  Find the values of x and y in x+y=8
0.7382


In [24]:
cos_ = torch.cosine_similarity(word_embeddings[0], word_embeddings[2], dim=0).item()
print(f'Cosine sim. entre "values" de')
print(f"  {input_strings[0]}")
print(f"  {input_strings[2]}")
print(f"{cos_:.4f}")

Cosine sim. entre "values" de
  "I voted for Obama because he was most aligned with my values", Mary said.
  I believe in the values of liberal democracy.
0.9002


**PREGUNTA** ¿Por qué obtenemos un valor más alto en el segundo caso que en el primero?

De acuerdo a [What Does BERT Look At? (Clark et al, 2019)](https://arxiv.org/abs/1906.04341) las correferencias tienden a estar captadas en los heads 4-5.


In [25]:
# attention from one token (left) to another (right)
tokens = bert_tokenizer.convert_ids_to_tokens(model_inputs.input_ids[0])
head_view(model_output.attentions, tokens)

<IPython.core.display.Javascript object>

¿Por qué `[SEP]` recibe tanta atención?

Una hipótesis es que funciona como un default cuando no aplica una función de un head (por ej, si un head representa objetos directos que prestan atención a verbos, tal vez los sustantivos en este head prestan atención a [SEP]).

En definitiva, para hacer análisis, a veces conviene no tener en cuenta este token.

Ver https://arxiv.org/pdf/1906.04341.pdf.

In [26]:
# BertViz show() está buenisimo pero no funciona bien para cualquier modelo
# ver https://colab.research.google.com/drive/1hXIQ77A4TYS4y3UthWF-Ci7V7vVUoxmQ?usp=sharing#scrollTo=-QnRteSLP0Hm

## Referencias

Generales:

* [HuggingFace Docs](https://huggingface.co/docs/transformers/index)
* [HuggingFace Course](https://huggingface.co/course/)
* [HuggingFace Book](https://transformersbook.com/) (Tunstall et al, 2022)

Específicas:

* HuggingFace tutorial de [Stanford CS224n](http://web.stanford.edu/class/cs224n/)
* [Entrena tu propio tokenizer](https://huggingface.co/docs/tokenizers/quicktour)
* [Carga tu propio dataset](https://huggingface.co/docs/datasets/loading)
* [Streaming de large datasets](https://huggingface.co/course/chapter5/4?fw=pt)
* [HF pipeline overview](https://huggingface.co/course/chapter2/2?fw=pt)
